# 06 Robustness Checks
In/out-of-sample splits and small parameter grids to check stability (not to optimize).

In [ ]:
import sys
from pathlib import Path

import pandas as pd
import matplotlib.pyplot as plt

repo_root = Path.cwd().resolve()
if not (repo_root / 'src').exists():
    repo_root = repo_root.parent
if str(repo_root) not in sys.path:
    sys.path.append(str(repo_root))

from src.data.etf_loader import load_clean_prices
from src.data.macro_loader import load_tnx_10y, load_vix
from src.signals.regime import compute_monthly_features, classify_regime
from src.signals.ls_biotech_pharma import compute_spread_momentum
from src.portfolio.vol_target import estimate_rolling_vol
from src.analysis.robustness_extension import sweep_rotation_parameters, sweep_regime_ls_parameters

%matplotlib inline


### Prepare data and labels


In [ ]:
prices = load_clean_prices().dropna()
# Regime labels
tnx = load_tnx_10y(start=prices.index.min().strftime('%Y-%m-%d'), end=prices.index.max().strftime('%Y-%m-%d')).reindex(prices.index).ffill()
vix = load_vix(start=prices.index.min().strftime('%Y-%m-%d'), end=prices.index.max().strftime('%Y-%m-%d')).reindex(prices.index).ffill()
features = compute_monthly_features(tnx, prices['SPY'], vix)
regime_labels = classify_regime(features)
spread_mom = compute_spread_momentum(prices[['XBI','XPH']], lookback_months=6)
vol_df = estimate_rolling_vol(prices[['XBI','XPH']].pct_change().fillna(0.0))


### Rotation parameter grid (small)
Check sensitivity on lookback, top_k, TS gating, target vol, leverage cap.


In [ ]:
rot_grid = sweep_rotation_parameters(
    prices=prices[['XBI','XPH','IHF','IHI','XLV']],
    lookbacks=[6, 12],
    top_ks=[1, 2],
    use_ts_flags=[True, False],
    target_vols=[0.10, 0.15],
    max_gross_list=[1.0, 1.5],
    transaction_cost_bps=10.0,
    split_date='2015-01-01',
)
rot_grid.sort_values('sharpe_out', ascending=False).head()


### Regime LS parameter grid (small)
Vary spread momentum threshold and gross exposure.


In [ ]:
ls_grid = sweep_regime_ls_parameters(
    prices=prices[['XBI','XPH']],
    vol_df=vol_df,
    regime_labels=regime_labels,
    spread_momentum=spread_mom,
    spread_mom_thresholds=[0.0, 0.02],
    target_gross_list=[0.8, 1.0],
    transaction_cost_bps=10.0,
    split_date='2015-01-01',
)
ls_grid.sort_values('sharpe_out', ascending=False).head()


### Commentary
- Grids are small by design; goal is to check stability, not hyper-optimize.
- Prefer parameter sets with reasonable in-sample and out-of-sample Sharpe, not just one side.
- If out-of-sample Sharpe collapses when nudging parameters, be cautious about overfitting.
